Conectando ao banco de dados SQLite.

In [31]:
## Forma mais direta
import sqlite3
connection = sqlite3.connect("pizza_app.sqlite")

Definindo uma função robusta para executar nossas queries SQL e pegar os erros pelo caminho.

In [32]:
from sqlite3 import Error

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)

        # commit necessário para alterações no banco
        connection.commit() ###

        print(f"Query executada.")
        if cursor.rowcount != -1:
            print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}'")

Criando as tabelas essenciais para o nosso sistema de pedidos de pizza.

In [33]:
# Cria a tabela produto #
create_produto_table = \
"""CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
);"""

execute_query(connection, create_produto_table)
#########################

# Cria a tabela pedido #
create_pedido_table = \
"""CREATE TABLE pedido (
    id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT
);"""

execute_query(connection, create_pedido_table)
#########################

# Cria a tabela item_pedido #
create_item_pedido_table = \
"""CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
);"""
execute_query(connection, create_item_pedido_table)
#########################

Erro: 'table produto already exists'
Erro: 'table pedido already exists'
Erro: 'table item_pedido already exists'


Inserindo manualmente alguns registros para dar vida às nossas tabelas.

In [34]:
## Inserindo registros manualmente

# Inserindo produto #
insert_produto = \
"""INSERT INTO
produto (tipo, desc_item, vl_preco)
VALUES
('ingrediente', 'camarão', 6),
('massa', 'tradicional', 9.25),
('borda', 'tradicional', 0),
('queijo', 'muçarela', 4),
('bebida', 'refrigerante', 5);
"""
execute_query(connection, insert_produto)
######################

# Inserindo pedido
insert_pedido = \
"""INSERT INTO
pedido (dt_pedido, fl_ketchup, desc_uf, txt_recado)
VALUES
('2023-06-01', TRUE, 'MG', 'Capricha no queijo!');
"""
execute_query(connection, insert_pedido)
######################

Query executada.
5 linha(s) afetadas
Query executada.
1 linha(s) afetadas


Utilizando `executemany` para inserir vários itens de pedido de uma vez só!

In [35]:
# Inserindo item_pedido
itens = (
    {'id_pedido': 1, 'id_produto': 2, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 3, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 1, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 4, 'qtd': 2},
    {'id_pedido': 1, 'id_produto': 5, 'qtd': 3}
)

insert_item_pedido = \
"""INSERT INTO item_pedido (id_pedido, id_produto, quantidade)
VALUES(:id_pedido, :id_produto, :qtd);"""

cursor = connection.cursor()
cursor.executemany(insert_item_pedido, itens)
connection.commit() # necessário para inserções
cursor.close()

Criando uma função para ler dados do nosso banco de dados.

In [36]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

Lendo todos os registros da tabela de produtos para ver o que temos em estoque.

In [37]:
tabela = 'produto'
query = f"SELECT * FROM {tabela}"
resultado = execute_read_query(connection, query)

print(f"Tabela: {tabela}")
for res in resultado:
    print(res)

Tabela: produto
(16, 'ingrediente', 'camarão', 6)
(17, 'massa', 'tradicional', 9.25)
(18, 'borda', 'tradicional', 0)
(19, 'queijo', 'muçarela', 4)
(20, 'bebida', 'refrigerante', 5)


Deletando todos os registros das nossas tabelas para preparar o banco para novos dados.

In [38]:
execute_query(connection, "DELETE FROM item_pedido;")
execute_query(connection, "DELETE FROM pedido;")
execute_query(connection, "DELETE FROM produto;")

Query executada.
5 linha(s) afetadas
Query executada.
1 linha(s) afetadas
Query executada.
5 linha(s) afetadas


Baixando os arquivos CSV com os dados da pizza para carregar no banco.

In [39]:
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/item_pedido.csv
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/pedido.csv
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/produto.csv

--2025-07-16 13:18:37--  https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/item_pedido.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103557 (101K) [text/plain]
Saving to: ‘item_pedido.csv’

item_pedido.csv     100%[===================>] 101.13K  --.-KB/s    in 0.01s   

2025-07-16 13:18:37 (9.24 MB/s) - ‘item_pedido.csv’ saved [103557/103557]

--2025-07-16 13:18:37--  https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/pedido.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting

Carregando os dados de pedidos de pizza do CSV para um DataFrame Pandas.

In [40]:
import pandas as pd
df_pedido = pd.read_csv(f'pedido.csv')
display(df_pedido.head())

,id_pedido,dt_pedido,fl_ketchup,desc_uf,txt_recado
0,0,2023-05-11,NaN,GO,NaN
1,1,2023-05-11,NaN,PR,Aquela pizza perfeita! :-D
2,2,2023-05-11,NaN,SP,Muito obrigado!!
3,3,2023-05-11,NaN,SP,NaN
4,4,2023-05-11,NaN,RS,Capricha no peperoni


Adicionando os dados do DataFrame de pedidos à tabela SQL 'pedido'.

In [41]:
df_pedido.to_sql('pedido', connection, if_exists='append', index=False)

count_rows = "SELECT COUNT(id_pedido) as count_id FROM pedido;"
print(execute_read_query(connection, count_rows))

select_all = f"SELECT * FROM pedido WHERE id_pedido < 5;"
execute_read_query(connection, select_all)

[(1106,)]


[(0, '2023-05-11', None, 'GO', None),
 (1, '2023-05-11', None, 'PR', 'Aquela pizza perfeita! :-D'),
 (2, '2023-05-11', None, 'SP', 'Muito obrigado!!'),
 (3, '2023-05-11', None, 'SP', None),
 (4, '2023-05-11', None, 'RS', 'Capricha no peperoni')]

Substituindo a tabela de pedidos e verificando a estrutura do esquema do banco.

In [42]:
df_pedido.to_sql('pedido', connection, if_exists='replace', index=False)
res = execute_read_query(connection, "SELECT sql FROM sqlite_schema")
for r in res:
    print(r[0])

CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
)
CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
)
None
CREATE TABLE "pedido" (
"id_pedido" INTEGER,
  "dt_pedido" TEXT,
  "fl_ketchup" INTEGER,
  "desc_uf" TEXT,
  "txt_recado" TEXT
)


Consultando os pedidos por estado para descobrir de onde vêm mais amantes de pizza!

In [43]:
query="""
SELECT desc_uf, COUNT(*) as count_pedidos
FROM pedido
GROUP BY desc_uf
ORDER BY count_pedidos DESC
LIMIT 5
"""
pd.read_sql_query(query, connection)

,desc_uf,count_pedidos
0,SP,395
1,RJ,103
2,MG,92
3,PR,76
4,RS,50


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['count_pedidos'].plot(kind='hist', bins=20, title='count_pedidos')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('desc_uf').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='index', y='count_pedidos', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7['count_pedidos'].plot(kind='line', figsize=(8, 4), title='count_pedidos')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['desc_uf'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='desc_uf', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['desc_uf'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='count_pedidos', y='desc_uf', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)